In [6]:
import os
# 关键：设置 Hugging Face 国内镜像站
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# --- 然后才是原本的代码 ---
import numpy as np
import torch
from sentence_transformers import SentenceTransformer

In [8]:
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from deepctr_torch.inputs import SparseFeat, VarLenSparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import DIN, DeepFM

In [18]:
import sys
# 使用清华镜像源快速安装
!{sys.executable} -m pip install seaborn -i https://pypi.tuna.tsinghua.edu.cn/simple

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [7]:
!pip install torch-rechub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torch-rechub]


In [6]:
!pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pymilvus]


In [8]:
!pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.9 MB/s  0:00:01 eta 0:00:01


In [4]:
# 检查 GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cpu


# 1. 数据加载与预处理 (MIND)——数据清洗与特征工程

#### 两张表：new表、behaviors表（谁在什么背景下做了选择）
##### 唯一标识 (news_id)

##### behaviors表：上下文/长期兴趣 (history)：
一串空格分隔的 news_id（如 N8668 N39081...），这是用户在这次曝光之前点击过的所有新闻。这是模型用来学习“用户喜欢什么”的主要依据。
##### behaviors表： 预测目标/当前候选集 (impressions)：
是这种格式：N123-1 N456-0 N789-0 ...（新闻ID-点击标签），我们需要把它 “炸开” (Explode)，变成一行一条样本：(User, History, Target_News, Label)。

In [4]:
# 定义 MIND 数据集的列名
news_cols = ['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities']
behaviors_cols = ['impression_id', 'user_id', 'time', 'history', 'impressions']
# 读取训练数据
train_news = pd.read_csv('data/MINDlarge_train/news.tsv', sep='\t', header=None, names=news_cols)
train_behaviors = pd.read_csv('data/MINDlarge_train/behaviors.tsv', sep='\t', header=None, names=behaviors_cols)
# 读取验证数据
valid_news = pd.read_csv('data/MINDlarge_dev/news.tsv', sep='\t', header=None, names=news_cols)
valid_behaviors = pd.read_csv('data/MINDlarge_dev/behaviors.tsv', sep='\t', header=None, names=behaviors_cols)
# 合并新闻数据（drop_duplicates去重）
all_news = pd.concat([train_news, valid_news]).drop_duplicates(subset=['news_id'])

print("数据加载完成！")
print(f"总新闻数: {len(all_news)}")
print(f"训练行为数: {len(train_behaviors)}")
print(f"验证行为数: {len(valid_behaviors)}")

数据加载完成！
总新闻数: 104151
训练行为数: 2232748
验证行为数: 376471


In [16]:
print("train_news表")
print(train_news.head(5))
print("train_behaviors表")
print(train_behaviors.head(5))
print("all_news表")
print(all_news.head(5))

train_news表
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
  news_id   category               subcategory  \
0  N88753  lifestyle           lifestyleroyals   
1  N45436       news  newsscienceandtechnology   
2  N23144     health                weightloss   
3  N86255     health                   medical   
4  N93187       news                 newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1    Walmart Slashes Prices on Last-Generation iPads   
2                      50 Worst Habits For Belly Fat   
3  Dispose of unwanted prescription drugs during ...   
4  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Apple's new iPad releases bring big deals on l...   
2  These seemingly harmless 

### 第 1 步：数据清洗与样本构建——负采样和样本展平

这一步我们将把“一次曝光（Session）”拆解成“多个训练样本”。为了防止内存爆炸，我们在这里会做一个 负采样 (Negative Sampling)：对于每个点击（正样本），我们随机选 4 个没点击的（负样本）。

#### 输入：原始 behaviors 表，输出：一个新的 DataFrame
将原始behaviors表的impressions“一个样本(一行)代表一次曝光(包含多个新闻)”格式转换为“一个样本(一行)代表一对User-Item关系”，并且在训练集中强行控制正负样本的比例。

##### 训练集策略：扩充正样本，随机采样负样本。强制实现 1 个正样本搭配 neg_ratio (4) 个负样本。
##### 验证集策略：全量保留。不丢弃任何数据，还原真实场景用于评估。
验证集模拟真实世界。在真实世界推荐中，我们不能故意把负样本扔掉，我们需要评估模型在面对大量负样本时，能否依然把正样本排在前面。因此，所有曝光数据都要保留

In [5]:
import random
from tqdm import tqdm

def process_mind_data_ratio(df, mode='train', neg_ratio=4):
    """
    升级版数据处理：
    1. 保持正负样本 1:N=neg_ratio 的比例 (通常 1:4 效果最好)
    2. 让模型见识更多负样本，提升判别能力
    """
    samples = []
    
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing {mode}"):
        user_id = str(row['user_id'])    
        # 处理历史记录history：如果是缺失值 NaN（空值），填补为 'N0'（防止报错）；否则按空格切分再合并
        hist = str(row['history']).split() if pd.notna(row['history']) else ['N0'] # hist = ['N123', 'N456', 'N789'] 或者['N0']
        hist_str = " ".join(hist) # hist_str = "N123 N456 N789" 或者 "N0"  ->将hist列表中所有元素连结起来生成一个字符串，中间用空格隔开
        # 处理impressions String->List (impressions = "N78206-0 N26368-0 N7578-1")
        impressions = str(row['impressions']).split() #impressions = ['N78206-0', 'N26368-0', 'N7578-1']
        
        pos_list = [] #正样本列表
        neg_list = [] #负样本列表

        # 1. 先把本次曝光的所有文章按正负分类
        for imp in impressions: # 遍历该用户这一次刷到的所有新闻 #impressions = ['N78206-0', 'N26368-0', 'N7578-1'] 或者 ['N78206', 'N7578-1']
            if '-' not in imp: continue    #跳过['N78206']无效记录
            news_id, label = imp.split('-')# "N78206-0" -> ['N78206','0'], news_id = "N78206" ,label = "0"
            label = int(label)  # "0"-> 0
            if label == 1:
                pos_list.append(news_id) # 用户点的
            else:
                neg_list.append(news_id) # 用户没点的
        
        # 2. 构建训练样本 （训练集核心策略：1:4 负采样）
        if mode == 'train':
            for pos_news in pos_list:
                # 加入正样本
                samples.append([user_id, hist_str, pos_news, 1])
                
                # 负采样：为每个正样本配 N=neg_ratio 个负样本
                # 如果负样本不够，就重复采样；如果够，就随机采不重复的
                if len(neg_list) >= neg_ratio:
                    negs = random.sample(neg_list, neg_ratio)
                else:
                    # 负样本不够，循环填充 （整数倍复制 {neg_ratio // len(neg_list)} + 余数补齐{:neg_ratio % len(neg_list)}）
                    negs = neg_list * (neg_ratio // len(neg_list)) + neg_list[:neg_ratio % len(neg_list)]
                
                for neg_news in negs:
                    samples.append([user_id, hist_str, neg_news, 0])
        else:
            # 验证集/测试集：保留所有数据，不做采样
            for imp in impressions:
                if '-' not in imp: continue
                news_id, label = imp.split('-')
                samples.append([user_id, hist_str, news_id, int(label)])
            
    return pd.DataFrame(samples, columns=['user_id', 'history_str', 'news_id', 'label'])

# --- 重新生成数据 ---
print("正在重新处理数据 (1:4 采样)...")
# 使用全量数据
train_df = process_mind_data_ratio(train_behaviors, mode='train', neg_ratio=4)
val_df = process_mind_data_ratio(valid_behaviors, mode='valid') # 验证集保持原样

print(f"\n最终训练集大小: {len(train_df)}")
print("训练集标签分布 (应接近 1:4):")
print(train_df['label'].value_counts())

正在重新处理数据 (1:4 采样)...


Processing valid: 100%|██████████████| 376471/376471 [00:17<00:00, 21518.08it/s]



最终训练集大小: 16918280
训练集标签分布 (应接近 1:4):
label
0    13534624
1     3383656
Name: count, dtype: int64


### 第 2 步：ID 编码 (Label Encoding)

我们需要把 User ID 和 News ID 转换成数字索引。

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --- 1. 编码 User ID ---
#标签编码LabelEncoder，给每个用户分配一个唯一的整数（0, 1, 2...）
lbe_user = LabelEncoder() #sklearn.preprocessing.LabelEncoder 的一个实例
all_users = pd.concat([train_df['user_id'], val_df['user_id']]).unique() #把训练集和验证集的所有用户 ID 拼在一起去重，建立一个“用户花名册” all_users
lbe_user.fit(all_users) #对所有用户花名册all_users 应用 LabelEncoder，给每个用户分配了一个唯一的整数编号（从 0 开始）

train_df['user_id_idx'] = lbe_user.transform(train_df['user_id'])
val_df['user_id_idx'] = lbe_user.transform(val_df['user_id'])

# --- 2. 编码 News ID ---
#模型需要一个包含所有可能出现的新闻的字典。
lbe_news = LabelEncoder #这是一个 LabelEncoder 实例

all_news_ids = set(train_df['news_id']) | set(val_df['news_id']) #测试集和验证集找新闻id
for h_str in train_df['history_str']: #在用户的历史记录里找新闻id
    all_news_ids.update(h_str.split())
all_news_ids.add('N0') #在新闻id中增加一个N0

lbe_news.fit(list(all_news_ids)) #LabelEncoder实例 lbe_news 记住了所有的去重后的新闻 ID，并编码 (0,1,2,...)
vocab_size_news = len(lbe_news.classes_) + 1 #所有新闻的总数量 + 一个N0占位符

# News ID 的映射字典： news_map = {"N001": 1,"N002":2,... }
news_map = dict(zip(lbe_news.classes_, range(1, len(lbe_news.classes_) + 1))) 
#生成的 news_map 是从 1 开始编号的（range(1, ...)）。这是为了把 0 号索引留给 Padding（填充值）

# --- 3. 建立“新闻 -> 类别”的查找表 ---
#处理 Category 和 SubCategory 

#现实数据往往是不完整的。有些新闻可能没有分类，我们不能留空，必须填一个默认值 'UNK' (Unknown)，否则后续编码会报错。
all_news['category'] = all_news['category'].fillna('UNK')
all_news['subcategory'] = all_news['subcategory'].fillna('UNK')

# 建立 NewsID -> Category 的映射表
lbe_cat = LabelEncoder()
lbe_subcat = LabelEncoder()
all_news['cat_idx'] = lbe_cat.fit_transform(all_news['category']) + 1 # 0做padding
all_news['subcat_idx'] = lbe_subcat.fit_transform(all_news['subcategory']) + 1

vocab_size_cat = len(lbe_cat.classes_) + 1 #category的类别数量+一个N0占位符
vocab_size_subcat = len(lbe_subcat.classes_) + 1 #subcategory的类别数量+一个N0占位符

# 创建快速查找字典（映射字典）: news2cat和news2subcat
#NewsID_String -> Cat_Int
news2cat = dict(zip(all_news['news_id'], all_news['cat_idx']))
news2subcat = dict(zip(all_news['news_id'], all_news['subcat_idx']))
# 还要加上 N0 (padding)
news2cat['N0'] = 0
news2subcat['N0'] = 0

# 辅助函数：把 NewsID 列表转换为 Feature 列表
def map_ids_to_features(id_list, mapping_dict):
    return [mapping_dict.get(x, 0) for x in id_list]
    #这是为了处理**“未知/冷启动”**物品。如果某个 ID 在字典里找不到（例如测试集里的新新闻），它会返回默认值 0。


# 编码训练数据、测试数据的new_id_idx
train_df['news_id_idx'] = train_df['news_id'].apply(lambda x: news_map.get(x, 0))
val_df['news_id_idx'] = val_df['news_id'].apply(lambda x: news_map.get(x, 0))

# 编码训练数据、测试数据的类别/子类别特征
train_df['cat_idx'] = train_df['news_id'].apply(lambda x: news2cat.get(x, 0))
val_df['cat_idx'] = val_df['news_id'].apply(lambda x: news2cat.get(x, 0))
train_df['subcat_idx'] = train_df['news_id'].apply(lambda x: news2subcat.get(x, 0))
val_df['subcat_idx'] = val_df['news_id'].apply(lambda x: news2subcat.get(x, 0))

## ————构建用户历史行为的多模态序列特征——————用户的一串新闻 ID 历史，转化成了三串平行的序列：ID 序列、类别序列和子类别序列。

def get_seq_features(hist_str, mapping_dict): 
    """
    输入：用户历史新闻记录、新闻类别字典（一级类别/二级类别）
        hist_str: 用户的历史记录字符串，例如 "N123 N456 N789"
        mapping_dict: 一个映射字典。比如 news2cat（新闻ID -> 类别ID）
    输出：用户历史新闻记录对应的新闻类别列表（一级类别/二级类别）
    ['N123', 'N456', 'N789'] -> [5, 12, 3]
    """
    raw_ids = hist_str.split()  #把字符串切分成列表 ['N123', 'N456', 'N789']
    #遍历列表中的每个新闻 ID，去字典里查它的属性（类别或子类别）。如果查不到（比如是个新 ID），就返回 0
    feats = [mapping_dict.get(x, 0) for x in raw_ids] 
    return feats

# 训练集测试集中 用户历史行为History News ID编码
train_df['hist_idx_list'] = train_df['history_str'].apply(lambda x: [news_map.get(i, 0) for i in x.split()])
val_df['hist_idx_list'] = val_df['history_str'].apply(lambda x: [news_map.get(i, 0) for i in x.split()])

# 训练集测试集中 用户历史行为 History Category类别编码（一级类别）
train_df['hist_cat_list'] = train_df['history_str'].apply(lambda x: get_seq_features(x, news2cat))
val_df['hist_cat_list'] = val_df['history_str'].apply(lambda x: get_seq_features(x, news2cat))

# 训练集测试集中 用户历史行为 History SubCategory类别编码（一级类别）
train_df['hist_subcat_list'] = train_df['history_str'].apply(lambda x: get_seq_features(x, news2subcat))
val_df['hist_subcat_list'] = val_df['history_str'].apply(lambda x: get_seq_features(x, news2subcat))

## 用户历史行为的 Padding & Truncating 处理
#对变长的序列数据进行“标准化”处理，使其变成固定长度（Fixed Length），以便输入到深度学习模型（如 DIN、DeepFM）中。
#神经网络通常要求输入矩阵的维度是固定的，而用户的历史浏览记录有的长、有的短，所以必须进行 Padding（填充） 或 Truncating（截断）

SEQ_LEN = 20#定义序列的最大长度为 20
def pad_col(df, col_name):
    return list(pad_sequences(df[col_name].tolist(), maxlen=SEQ_LEN, padding='post', truncating='post', value=0))

train_df['history_seq'] = pad_col(train_df, 'hist_idx_list')
val_df['history_seq'] = pad_col(val_df, 'hist_idx_list')
train_df['hist_cat_seq'] = pad_col(train_df, 'hist_cat_list')
val_df['hist_cat_seq'] = pad_col(val_df, 'hist_cat_list')
train_df['hist_subcat_seq'] = pad_col(train_df, 'hist_subcat_list')
val_df['hist_subcat_seq'] = pad_col(val_df, 'hist_subcat_list')

print("特征工程升级完成：已加入类别特征！")

正在处理分类特征...
特征工程升级完成：已加入类别特征！


### 第 3 步：🌟 BERT语义特征提取 (简历核心亮点)

这是你简历中 "基于 BERT 提取文字向量" 的实现部分。我们将使用 sentence-transformers 来提取新闻标题的 Embedding，并构建一个 Embedding Matrix。

##### 利用预训练的 BERT 模型（大语言模型LLM）提取新闻标题的语义特征，并将其对齐到推荐系统的 ID 索引上，作为模型 Embedding 层的初始权重，用 BERT 读懂新闻标题，然后把读懂的知识（向量）塞给 DIN 模型，让它赢在起跑线上

In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from torch_rechub.utils.match import pad_sequences
# 配置

USE_LLM = True # 如果网络不好先设为 False

EMBEDDING_DIM = 32
# 1. 确保 vocab_size 正确 (包含 0 号 padding)
# 注意：lbe_news.classes_ 是所有新闻的原始 ID
vocab_size_news = len(lbe_news.classes_) + 1
print(f"News 词表大小: {vocab_size_news}")

# 2. 初始化 Embedding 矩阵 (全 0)
# 行数 = vocab_size, 列数 = EMBEDDING_DIM
pretrained_news_emb = np.zeros((vocab_size_news, EMBEDDING_DIM))

if USE_LLM:
    import os
    os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com' # 使用镜像
    from sentence_transformers import SentenceTransformer
    print("正在加载 BERT 模型...")
    model_bert = SentenceTransformer('all-MiniLM-L6-v2')

    
    # 获取所有新闻标题，保持顺序
    titles = all_news.set_index('news_id').reindex(lbe_news.classes_)['title'].fillna("").tolist()

    # 生成 Embedding
    embeddings = model_bert.encode(titles, batch_size=128, show_progress_bar=True)

    # 降维
    from sklearn.decomposition import PCA
    pca = PCA(n_components=EMBEDDING_DIM)
    embeddings_32 = pca.fit_transform(embeddings)

    
    # 【核心修正点】
    # lbe_news.classes_ 中的第 0 个新闻，对应的编码是 1 (因为我们在特征工程里 +1 了)
    # 所以 embeddings_32 的第 0 行，应该填入 pretrained_news_emb 的第 1 行
    pretrained_news_emb[1:] = embeddings_32
    print("BERT 向量注入完成 (Index 1 to N)")

else:
    print("使用随机初始化向量模拟 (演示模式)...")
    # 【核心修正点】从索引 1 开始填充，索引 0 保持为 0 (Padding)
    pretrained_news_emb[1:] = np.random.uniform(-0.1, 0.1, size=(vocab_size_news - 1, EMBEDDING_DIM))

# 转为 Tensor
pretrained_news_emb_tensor = torch.FloatTensor(pretrained_news_emb)
print(f"Embedding Matrix Shape: {pretrained_news_emb_tensor.shape}")

from torch_rechub.models.ranking import DIN
from torch_rechub.basic.features import SparseFeature, SequenceFeature
from torch_rechub.trainers import CTRTrainer
from torch_rechub.utils.data import DataGenerator
import torch


# 1. 定义特征列
user_cols = [SparseFeature("user_id_idx", vocab_size=len(lbe_user.classes_), embed_dim=32)]
# Item 特征 (News ID + Category + SubCategory)
item_cols = [
    SparseFeature("news_id_idx", vocab_size=vocab_size_news, embed_dim=32),
    # 【修改】，与 news_id 保持一致
    SparseFeature("cat_idx", vocab_size=vocab_size_cat, embed_dim=32),       
    SparseFeature("subcat_idx", vocab_size=vocab_size_subcat, embed_dim=32)
]


# 序列特征 (News ID + Category + SubCategory)
history_cols = [
    SequenceFeature("history_seq", vocab_size=vocab_size_news, embed_dim=32, pooling="concat", shared_with="news_id_idx"),
    # 【修改】
    SequenceFeature("hist_cat_seq", vocab_size=vocab_size_cat, embed_dim=32, pooling="concat", shared_with="cat_idx"),
    SequenceFeature("hist_subcat_seq", vocab_size=vocab_size_subcat, embed_dim=32, pooling="concat", shared_with="subcat_idx")
]

features = user_cols + item_cols



News 词表大小: 101288
正在加载 BERT 模型...


Batches:   0%|          | 0/792 [00:00<?, ?it/s]

BERT 向量注入完成 (Index 1 to N)
Embedding Matrix Shape: torch.Size([101288, 32])


### 第 4 步：构建 DIN 模型 (Torch-Rechub) AUC=0.712

这里我们要做一个关键操作：将我们生成的 pretrained_news_emb_tensor 注入到 DIN 模型中，作为新闻 ID 的 Embedding 初始化权重。这就在代码层面实现了“LLM 增强推荐”。

In [38]:
import torch
import os
import numpy as np

# ==========================================
# 2. 准备数据 (Mac 优化版)
# ==========================================
print("💻 正在构建 DataLoader (Mac 本地模式)...")

# 定义列
input_cols = ['user_id_idx', 'news_id_idx', 'cat_idx', 'subcat_idx', 'history_seq', 'hist_cat_seq', 'hist_subcat_seq']

train_dg = DataGenerator(x=train_df[input_cols], y=train_df['label'])
val_dg = DataGenerator(x=val_df[input_cols], y=val_df['label'])

# 【Mac 设置建议】
# 1. num_workers=0: Mac 在 Jupyter 里多进程容易报 BrokenPipe，设为 0 最稳。
# 2. batch_size=4096: Mac 内存通常统一，适当调大可以加速 (比 1024 快)。
train_loader, val_loader, _ = train_dg.generate_dataloader(
    x_val=val_df[input_cols], 
    y_val=val_df['label'], 
    batch_size=4096, 
    num_workers=0
)

# ==========================================
# 3. 定义模型 & 注入权重
# ==========================================
print("正在构建模型...")
model_din = DIN(
    features=features,
    history_features=history_cols,
    target_features=item_cols,
    mlp_params={"dims": [256, 128], "dropout": 0.3},
    attention_mlp_params={"dims": [64, 32]}
)

# --- 注入 BERT 权重 ---
target_layer_found = False
embedding_weight_param = None 

for name, param in model_din.named_parameters():
    if "news_id_idx" in name and "weight" in name:
        param.data.copy_(pretrained_news_emb_tensor)
        embedding_weight_param = param 
        target_layer_found = True
        print("✅ BERT Embedding 注入成功")
        break
if not target_layer_found:
    print("❌ 警告：未找到 Embedding 层！")

# ==========================================
# 4. 两阶段训练策略 (自动保存最佳模型)
# ==========================================
# Mac M系列芯片支持 mps 加速，Intel芯片用 cpu
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available(): 
    device = 'mps'
    print("🍎 检测到 MacOS MPS (Metal) 加速开启！")

# ------------------------------------
# 阶段一：Warm Up (冻结 BERT)
# ------------------------------------
print("\n🔥 [阶段一] 开始 Warm Up (冻结 BERT)...")

if embedding_weight_param is not None:
    embedding_weight_param.requires_grad = False

# Warmup 只跑 1 轮，不需要存模型，直接 fit
trainer_warmup = CTRTrainer(
    model=model_din,
    optimizer_params={"lr": 1e-3, "weight_decay": 1e-4},
    n_epoch=1, 
    device=device
)
trainer_warmup.fit(train_dataloader=train_loader, val_dataloader=val_loader)

# ------------------------------------
# 阶段二：Fine-tuning (解冻 BERT + 擂台赛保存法)
# ------------------------------------
import torch
import os

print("\n🚀 [阶段二] 开始 Fine-tuning (解冻 BERT，寻找最佳参数)...")

# 1. 解冻 Embedding
if embedding_weight_param is not None:
    embedding_weight_param.requires_grad = True

# 2. 定义优化器参数
# 注意：这里我们手动控制循环，所以 n_epoch 在 Trainer 里设为 1 即可，由外层循环控制总轮数
trainer_finetune = CTRTrainer(
    model=model_din,
    optimizer_params={"lr": 5e-5, "weight_decay": 1e-4}, 
    n_epoch=1, 
    device=device
)

# 3. 初始化最佳记录
best_auc = 0.0
best_epoch = 0
total_epochs = 10  # 计划跑 10 轮

for epoch in range(total_epochs):
    print(f"\n>>> Training Epoch {epoch + 1}/{total_epochs}...")
    
    # 1. 训练一轮
    # torch_rechub 的 train_one_epoch 通常不需要传 model，它用 self.model
    trainer_finetune.train_one_epoch(train_loader)
    
    # 2. 验证一轮 (获取当前 AUC)
    # 【核心修复】必须传入 model_din 作为第一个参数！
    current_auc = trainer_finetune.evaluate(model_din, val_loader)
    
    print(f"    Epoch {epoch + 1} Validation AUC: {current_auc:.5f}")
    
    # 3. 擂台赛：如果当前 AUC 超过历史最高
    if current_auc > best_auc:
        best_auc = current_auc
        best_epoch = epoch + 1
        
        # 保存这一轮的模型为 "最佳模型"
        torch.save(model_din.state_dict(), 'din_best_model.pth')
        print(f"    🏆 创新高！模型已保存 (AUC: {best_auc:.5f})")
    else:
        print(f"    (未超过最佳 AUC: {best_auc:.5f} @ Epoch {best_epoch})")

print("="*40)
print(f"🎉 训练结束！")
print(f"最佳模型出现在 Epoch {best_epoch}，AUC 为 {best_auc:.5f}")
print("你可以使用 model_din.load_state_dict(torch.load('din_best_model.pth')) 加载最佳状态。")

💻 正在构建 DataLoader (Mac 本地模式)...
正在构建模型...
✅ BERT Embedding 注入成功
🍎 检测到 MacOS MPS (Metal) 加速开启！

🔥 [阶段一] 开始 Warm Up (冻结 BERT)...
epoch: 0


validation: 100%|███████████████████████████| 3439/3439 [16:23<00:00,  3.50it/s]


epoch: 0 validation: auc: 0.6789376735356462

🚀 [阶段二] 开始 Fine-tuning (解冻 BERT，寻找最佳参数)...

>>> Training Epoch 1/10...


validation: 100%|███████████████████████████| 3439/3439 [16:38<00:00,  3.44it/s]


    Epoch 1 Validation AUC: 0.68580
    🏆 创新高！模型已保存 (AUC: 0.68580)

>>> Training Epoch 2/10...


...

    Epoch 8 Validation AUC: 0.70231
    (未超过最佳 AUC: 0.71329 @ Epoch 4)

>>> Training Epoch 9/10...


validation: 100%|███████████████████████████| 3439/3439 [12:01<00:00,  4.76it/s]


    Epoch 9 Validation AUC: 0.70299
    (未超过最佳 AUC: 0.71329 @ Epoch 4)

>>> Training Epoch 10/10...


validation: 100%|███████████████████████████| 3439/3439 [12:13<00:00,  4.69it/s]


    Epoch 10 Validation AUC: 0.70060
    (未超过最佳 AUC: 0.71329 @ Epoch 4)
🎉 训练结束！
最佳模型出现在 Epoch 4，AUC 为 0.71329
你可以使用 model_din.load_state_dict(torch.load('din_best_model.pth')) 加载最佳状态。


In [40]:
import numpy as np
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
model_din.load_state_dict(torch.load('din_best_model.pth'))
# ==========================================
# 5. 多维度评估函数 (适配 NumPy 2.0)
# ==========================================

def calculate_metrics(grouped_df):
    """
    计算单个曝光列表的指标
    """
    aucs, mrrs, ndcg5s, ndcg10s = [], [], [], []
    
    # 遍历每个用户的曝光列表
    for _, group in tqdm(grouped_df, desc="Evaluating Ranking Metrics"):
        labels = group['label'].values
        preds = group['pred'].values
        
        # 1. 如果全是负样本或全是正样本，无法计算 AUC，跳过
        if len(np.unique(labels)) == 1:
            continue
            
        # --- AUC ---
        try:
            aucs.append(roc_auc_score(labels, preds))
        except ValueError:
            continue
        
        # --- 排序预处理 ---
        # 根据预测分数从高到低排序
        sorted_indices = np.argsort(preds)[::-1]
        sorted_labels = labels[sorted_indices]
        
        # --- MRR (Mean Reciprocal Rank) ---
        first_pos_idx = np.where(sorted_labels == 1)[0]
        if len(first_pos_idx) > 0:
            mrrs.append(1.0 / (first_pos_idx[0] + 1))
        else:
            mrrs.append(0.0)
            
        # --- nDCG (Normalized Discounted Cumulative Gain) ---
        # 辅助函数：计算 DCG
        def dcg_at_k(r, k):
            # 【修复点】np.asfarray -> np.asarray(..., dtype=float)
            r = np.asarray(r, dtype=float)[:k]
            if r.size:
                return np.sum(r / np.log2(np.arange(2, r.size + 2)))
            return 0.

        # 辅助函数：计算 nDCG
        def ndcg_at_k(r, k):
            dcg_max = dcg_at_k(sorted(r, reverse=True), k)
            if not dcg_max:
                return 0.
            return dcg_at_k(r, k) / dcg_max

        ndcg5s.append(ndcg_at_k(sorted_labels, 5))
        ndcg10s.append(ndcg_at_k(sorted_labels, 10))
        
    return np.mean(aucs), np.mean(mrrs), np.mean(ndcg5s), np.mean(ndcg10s)

# ==========================================
# 6. 执行评估 (修复 predict 参数)
# ==========================================
print("\n📊 正在进行全量多维度评估...")

# 1. 获取模型预测分
model_din.eval() 

print("正在预测验证集分数...")
# 【注意】必须传入 model_din 作为第一个参数
y_pred = trainer_finetune.predict(model_din, val_loader) 

# 2. 将预测分拼回到验证集 DataFrame
eval_df = val_df.copy()
eval_df['pred'] = np.array(y_pred)

# 3. 确定分组列
if 'impression_id' not in eval_df.columns:
    print("⚠️ 警告：未检测到 impression_id，使用 user_id 近似分组")
    eval_df['group_id'] = eval_df['user_id'] 
else:
    eval_df['group_id'] = eval_df['impression_id']

# 4. 分组并计算
grouped = eval_df.groupby('group_id')
gauc, mrr, ndcg5, ndcg10 = calculate_metrics(grouped)

print("="*30)
print(f"✅ 最终评估结果 (Validation Set):")
print(f"Global AUC (LogLoss): {roc_auc_score(val_df['label'], eval_df['pred']):.4f}")
print(f"Group AUC (GAUC)    : {gauc:.4f}")
print(f"MRR                 : {mrr:.4f}")
print(f"nDCG@5              : {ndcg5:.4f}")
print(f"nDCG@10             : {ndcg10:.4f}")
print("="*30)


📊 正在进行全量多维度评估...
正在预测验证集分数...


predict: 100%|██████████████████████████████| 3439/3439 [11:17<00:00,  5.08it/s]


⚠️ 警告：未检测到 impression_id，使用 user_id 近似分组


Evaluating Ranking Metrics: 100%|█████| 255990/255990 [01:51<00:00, 2299.02it/s]


✅ 最终评估结果 (Validation Set):
Global AUC (LogLoss): 0.7133
Group AUC (GAUC)    : 0.6932
MRR                 : 0.3818
nDCG@5              : 0.3305
nDCG@10             : 0.3917


### DCN

In [58]:
from torch_rechub.models.ranking import DCN
from torch_rechub.trainers import CTRTrainer
import torch

# 1. 定义 DCN 模型
# DCN = Deep Network (DNN) + Cross Network (显式高阶交互)
# 核心参数是 n_cross_layers (交叉层数)，通常设为 2 或 3
model_dcn = DCN(
    features=features, 
    mlp_params={"dims": [256, 128], "dropout": 0.4}, # Deep 部分
    n_cross_layers=3,                                # Cross 部分 (3层交互)
)

# 2. 注入 LLM Embedding
# DCN 的 embedding 存储位置和 DIN 一样，也是在 model.embedding 中
target_layer_found = False

for name, param in model_dcn.named_parameters():
    # DCN 的结构里同样包含 embedding 层
    if "news_id_idx" in name and "weight" in name:
        print(f"Found layer: {name}")
        param.data.copy_(pretrained_news_emb_tensor)
        
        # 【策略】解冻，允许微调，让 LLM 向量适应 DCN 的交叉结构
        param.requires_grad = True 
        target_layer_found = True
        break

if target_layer_found:
    print("✅ LLM Embedding 已成功注入 DCN！")
else:
    print("❌ 注入失败")

# 3. 训练
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = 'mps'

trainer = CTRTrainer(
    model=model_dcn,
    # 沿用你调优过的参数
    optimizer_params={"lr": 5e-4, "weight_decay": 5e-4}, 
    n_epoch=3, 
    device=device
)

print("开始训练 DCN 模型 (1:4 Data + LLM + Categories)...")
trainer.fit(train_dataloader=train_loader, val_dataloader=val_loader)

Found layer: embedding.embed_dict.news_id_idx.weight
✅ LLM Embedding 已成功注入 DCN！
开始训练 DCN 模型 (1:4 Data + LLM + Categories)...
epoch: 0


validation: 100%|█████████████████████████████████████████████████████████████████████████████████████| 2677/2677 [00:51<00:00, 51.97it/s]


epoch: 0 validation: auc: 0.6168520333364222
epoch: 1


validation: 100%|█████████████████████████████████████████████████████████████████████████████████████| 2677/2677 [00:49<00:00, 53.78it/s]


epoch: 1 validation: auc: 0.630628156808087
epoch: 2


validation: 100%|█████████████████████████████████████████████████████████████████████████████████████| 2677/2677 [00:58<00:00, 46.09it/s]


epoch: 2 validation: auc: 0.6257678667263062


## 问题一：为什么 DCN 跑不过 DIN？

### 1.机制差异（Attention vs. Pooling）：

DIN 的核心优势：DIN 使用了 Attention（注意力机制）。当模型判断“要不要给用户推一条《湖人队比赛》新闻”时，DIN 会回头看用户的历史，自动高亮历史中和“篮球”相关的记录，忽略掉“理财”、“天气”等无关记录。这叫 Target-Aware（目标相关性）。

DCN 的劣势：DCN 虽然强在特征交叉，但它处理“历史行为序列”时，通常只能简单粗暴地把过去看过的 20 条新闻 取平均（Mean Pooling） 或拼接。这就把用户的兴趣“糊成了一团”。

结果：在这个新闻数据集上，“精准匹配兴趣”（DIN）比 “全局特征交叉”（DCN）更重要。

### 2.LLM 向量的利用率：

BERT 赋予了 Embedding 极强的语义（比如“苹果”和“iPhone”向量很近）。

DIN 的 Attention 机制直接计算 Target(苹果) 和 History(iPhone) 的向量相似度，完美利用了 BERT 的语义能力。

DCN 只是把这些向量当成普通数字去乘来乘去，浪费了 BERT 带来的语义对齐优势。

## 问题二：你提到的 COBRA 机制具体是怎么做的？你用的效果为什么不好

COBRA 论文提出生成式推荐需要结合 Sparse ID 和 Dense Vector。受此启发，我认为判别式推荐（CTR）也可以借鉴。我没有使用论文中复杂的 RQ-VAE 生成 ID ，而是工程化地使用了QA K-Means 对 BERT 向量进行聚类（尝试引入 RQ-KMeans (残差量化)，试图将新闻的 BERT 向量转化为离散的语义 ID（Codebook）。”），生成的 Cluster ID 作为一种‘语义 ID’输入到 DIN 的稀疏特征层，同时将 BERT 原始向量输入到稠密层。尝试了双路输入。

### 阶段一：离线准备（造“桶”）在模型训练之前，我们利用历史存量的文章（比如 10 万篇）做准备：

#### （1）BERT 向量化：Raw Text 新闻标题(人类看得懂，机器看不懂) -> Dense Vector 语义向量 (机器看懂了语义，变成了高维坐标)

把这 10 万篇文章的标题输入 BERT，得到 10 万个768维BERT向量。BERT 作为一个预训练大模型，它理解语义！！

#### （2）PCA降维：Dense Vector语义向量 -> embeddings_reduced (浓缩向量)

BERT 原生输出是 768 维。直接对 6.5 万条数据做 768 维的 K-Means 聚类计算量非常大，而且容易受“维度灾难”影响，效果不一定好。所以我们先用 PCA 把它浓缩成 64 维 (embeddings_reduced)。这既保留了核心语义，又让 K-Means 跑得更快、更准。

#### （3）QR K-Means 聚类：得到Sparse IDs (语义 ID)，根据坐标位置，给它贴上标签。

对这 10 万个64维向量embeddings_reduced进行聚类（假设 $K=1000$）。


保存中心点：我们得到了 32 个聚类中心（Centroids），并给它们编号 0 到 31。这些编号就是我们的 Semantic Cluster ID。

模型训练：在训练 DIN/DCN 时，我们将这个 Cluster_ID 作为一个特征输入。模型会学习到用户对这些“类簇”的喜好（例如：Cluster4代表“人工智能”，用户喜欢看）。

### 阶段二：冷启动推断（新文章来了！）现在，一篇关于“OpenAI 发布 Sora”的新闻来了（训练集中没见过）。

获取 Dense 特征：把标题扔进 BERT，直接算出它的向量 $V_{new}$。这就是 Dense Feature。它包含了“视频生成、AI”等细粒度语义。获取 Sparse ID 特征：计算 $V_{new}$ 与之前保存的 32 个聚类中心的距离。发现它离 Cluster 42（人工智能类）最近。直接赋予它 ID = 42。这就是 Sparse ID Feature。

模型推理：虽然模型没见过这篇文章的 Item_ID（此时通常用 UNK 或 Padding 处理，信息量为 0），但模型见过 ID=4。模型心里想：“哦，虽然不知道这是啥新闻，但它属于第 4 类，这个用户以前老看 4 类的新闻，所以我应该推荐它！”

#### 遇到的挑战：
实验中我发现，直接使用 RQ-KMeans 生成的离散 ID 进行训练，AUC 指标并不理想（徘徊在 0.5~0.6）。经过分析，我认为原因在于：硬量化（Hard Quantization）导致了严重的信息损失。将丰富的高维 BERT 语义强行压缩为几个离散的聚类中心，丢失了新闻标题中微妙的语义差异（Fine-grained Semantics）。

#### 改进和创新：
针对 CTR 预估这种对精度要求极高的排序（Ranking）任务，我调整了策略。我保留了 COBRA 的核心思想——‘引入外部语义知识’，但摒弃了它的‘离散化’过程。 我改用 BERT + PCA 降维 + Embedding Warm-up 的方案。这种**‘连续语义注入’**策略，既保留了预训练模型的高维语义空间结构，又避免了量化带来的精度损失。实验证明，这种改进方案将 AUC 提升到了 0.71。

### 为什么 RQ-KMeans 在你的实验中效果不好
COBRA/RQ-VAE 的主场：通常用于**召回（Retrieval）**阶段。召回需要在百万级数据里快速捞出 100 个候选，所以需要离散索引（Indexable）来加速。这时候牺牲一点精度换取速度是值得的。

DeepFM/DIN 是**精排（Ranking）**模型。它的任务是给这 100 个候选打分，决定谁排第一。排序对精度极其敏感。

K-Means 把“科比”和“詹姆斯”都聚类为“ID_5（篮球球星）”，这对召回够用了，但对排序来说，这种粒度太粗（Coarse-grained），模型无法区分两者细微的差别，导致 AUC 上不去

## 问题三：你的模型是如何处理冷启动问题的？
“我的模型主要通过**基于内容的语义迁移（Content-Based Semantic Transfer）和层级特征泛化（Hierarchical Feature Generalization）**来解决冷启动问题。”

### 1. 利用 BERT 实现“语义迁移”
对于没有交互行为的新闻（Item Cold Start），我利用 BERT 预训练模型提取标题的语义向量作为 Embedding 的初始化权重。这使得新物品在进入模型的第一刻，就拥有了具备实际语义的向量表示，而不是随机噪声。模型能直接根据语义相似度，将其推荐给喜欢同类内容的用户

### 2. 利用类别特征实现“层级泛化”
ID 是具体的，但类别是通用的。新新闻 N99999 虽然没见过，但它的类别 Sports 和子类别 Basketball 是模型已经在训练集中学得很好的。 你的做法： 你的模型输入中显式包含了 cat_idx (类别) 和 subcat_idx (子类别)。“除了 ID 特征，我还引入了 Category 和 SubCategory 特征。当新物品 ID 缺乏协同过滤信号时，DIN 模型可以通过用户对‘类别’的历史偏好（Attention 机制），降级匹配到该物品的类别特征上，从而保证推荐的准确性。”

### 3. 处理“用户冷启动”
对于新用户（User Cold Start），因为没有历史行为序列，模型会使用 Padding Token (N0) 进行填充。此时 DIN 的 Attention 机制失效，模型会退化为依据全局偏差（Bias）和物品的基础特征进行推荐，这实际上类似于向新用户推荐系统中的热门内容（Popularity-based），保证了基础的兜底体验。

目前对于完全无记录的新用户，我是用 Padding 处理的。未来如果上线，我会考虑在这个阶段加入基于规则的热门推荐或者基于用户注册画像（年龄、性别）的粗粒度推荐来进一步优化用户冷启动体验。

## 问题四：你的方法比传统的 DeepFM 或 W&D 好在哪里？
传统的深度推荐模型通常依赖 ID Embedding 的随机初始化，新物品必须积累大量点击（梯度更新）后才能学好向量。 而我的方案结合了 LLM (BERT)，实现了 Zero-Shot（零样本） 或 Few-Shot（少样本） 的启动能力。新物品一上架，模型就知道它是讲什么的，直接就能推荐给感兴趣的人，极大缩短了冷启动的预热周期。


#### 注意力机制可视化图


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import random
import platform
import numpy as np
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
# ============================
# 字体设置
# ============================
system_name = platform.system()
if system_name == "Darwin": 
    plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'PingFang HK', 'Heiti TC']
else: 
    plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False 

def visualize_din_attention_final(model, df, sample_index, news_map, all_news_df):
    """
    【最终完美版】可视化 DIN 注意力权重
    特性：
    1. 包含新闻类别标签 [Category]
    2. 修复所有显示和计算Bug
    """
    model.eval()
    device = next(model.parameters()).device
    
    # 检查索引是否存在
    if sample_index not in df.index:
        print(f"❌ 错误：索引 {sample_index} 不在当前验证集中。请检查索引号是否正确。")
        return

    # --- 1. 数据准备 ---
    row = df.loc[sample_index] # 使用 loc 以支持特定索引号
    target_id = int(row['news_id_idx'])
    
    inv_news_map = {v: k for k, v in news_map.items()}
    target_news_raw_id = inv_news_map.get(target_id, "UNK")
    
    # 获取 Target 新闻的详细信息 (标题 + 类别)
    try:
        # 查找对应的行
        tgt_info = all_news_df[all_news_df['news_id'] == target_news_raw_id].iloc[0]
        t_cat = tgt_info['category']
        t_sub = tgt_info['subcategory']
        t_title = tgt_info['title']
        # 格式化标题：【体育/篮球】湖人队赢了...
        target_display = f"【{t_cat}/{t_sub}】\n{t_title}"
    except:
        target_display = f"未知新闻 (ID: {target_news_raw_id})"

    # 解析历史序列
    hist_seq = row['history_seq'] 
    if isinstance(hist_seq, str): 
        hist_seq = eval(hist_seq)
    valid_hist_ids = [int(idx) for idx in hist_seq if idx != 0]
    
    if len(valid_hist_ids) == 0:
        print("⚠️ 该用户没有有效历史行为。")
        return

    # 获取 History 新闻的详细信息
    hist_labels = []
    for idx in valid_hist_ids:
        try:
            raw_id = inv_news_map.get(idx, "UNK")
            h_info = all_news_df[all_news_df['news_id'] == raw_id].iloc[0]
            
            h_cat = h_info['category']
            h_title = h_info['title']
            
            # 格式化历史标签: [体育] 标题...
            # 截断标题防止过长
            short_title = h_title[:70] + "..." if len(h_title) > 70 else h_title
            label = f"[{h_cat}] {short_title}"
            hist_labels.append(label)
        except:
            hist_labels.append("Unknown Item")

    # --- 2. 提取权重 ---
    try:
        emb_layer = model.embedding.embed_dict['news_id_idx']
        attn_layer = model.attention_layers[0] 
    except:
        print("❌ 模型结构不匹配，无法提取层。")
        return
    
    # --- 3. 计算分数 (BatchNorm修复版) ---
    with torch.no_grad():
        target_emb = emb_layer(torch.tensor([target_id]).to(device)) 
        keys_emb = emb_layer(torch.tensor([valid_hist_ids]).to(device)) 
        query_emb = target_emb.expand(keys_emb.shape[0], keys_emb.shape[1], -1)
        
        din_input = torch.cat([query_emb, keys_emb, query_emb - keys_emb, query_emb * keys_emb], dim=-1)
        
        # Flatten
        seq_len = din_input.shape[1]
        dim = din_input.shape[2]
        din_input_flat = din_input.view(-1, dim) 
        
        scores_flat = attn_layer.attention(din_input_flat) 
        attention_weights = scores_flat.squeeze().cpu().numpy()
        
        if attention_weights.ndim == 0:
            attention_weights = np.array([attention_weights])

    # --- 4. 绘图 ---
    # 高度根据历史记录数量自动调整
    fig, ax = plt.subplots(figsize=(13, len(valid_hist_ids) * 0.7 + 2))
    
    w_min, w_max = attention_weights.min(), attention_weights.max()
    norm_weights = (attention_weights - w_min) / (w_max - w_min + 1e-9)
    colors = plt.cm.coolwarm(norm_weights)
    
    y_pos = range(len(valid_hist_ids))
    ax.barh(y_pos, attention_weights, color=colors, height=0.6)
    
    # 设置左侧标签
    ax.set_yticks(y_pos)
    ax.set_yticklabels(hist_labels, fontsize=11, family='Arial Unicode MS') # 强制指定支持中文的字体
    
    ax.set_xlabel('Attention Score (关联度)', fontsize=12)
    # 标题左对齐显示
    ax.set_title(f'DIN 模型注意力透视\n候选目标: {target_display}', fontsize=14, color='darkblue', fontweight='bold', loc='left', pad=20)
    
    for i, v in enumerate(attention_weights):
        ax.text(v, i, f" {v:.2f}", va='center', fontsize=10, fontweight='bold')
        
    # 调整边距：左边留出 40% 给带类别的长标题
    plt.subplots_adjust(left=0.4, right=0.95, top=0.85, bottom=0.1)
    
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.show()

# ================================
# 🚀 运行 (固定 Index: 2694003)
# ================================
target_idx = 2694003

print(f"正在分析固定样本 Index: {target_idx} ...")
visualize_din_attention_final(
    model=model_din, 
    df=val_df, 
    sample_index=target_idx, 
    news_map=news_map, 
    all_news_df=all_news
)

正在分析固定样本 Index: 2694003 ...


NameError: name 'model_din' is not defined

#### 上传结果

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import zipfile
from tqdm import tqdm
from scipy.stats import rankdata
from torch_rechub.models.ranking import DIN
from torch_rechub.basic.features import SparseFeature, SequenceFeature
from torch_rechub.utils.data import DataGenerator
from tensorflow.keras.preprocessing.sequence import pad_sequences

# ==========================================
# 1. 强制对齐配置 (根据你的报错信息硬编码)
# ==========================================
EMBEDDING_DIM = 32
device = 'cpu' 

# ⚠️ 关键修改：直接使用报错信息里 "checkpoint" 的数值
# User: copying a param with shape torch.Size([750434, 32])
FIXED_USER_VOCAB = 750434 

# News: copying a param with shape torch.Size([101288, 32])
FIXED_NEWS_VOCAB = 101288 

# Cat/Subcat 没报错，说明维度没变，沿用之前计算的 (假设之前环境恢复代码已跑)
# 如果 cat_idx 也没对齐，需要把 vocab_size_cat 也硬编码，但通常这个比较稳
if 'vocab_size_cat' not in globals():
    # 万一内存空了，给个保底值 (MIND数据集通常 Cat<20, Subcat<300)
    vocab_size_cat = 30
    vocab_size_subcat = 300

print(f"🚀 强制对齐模型骨架: User={FIXED_USER_VOCAB}, News={FIXED_NEWS_VOCAB}")

# ==========================================
# 2. 重新定义模型骨架 (Features)
# ==========================================
# User 特征
user_cols = [SparseFeature("user_id_idx", vocab_size=FIXED_USER_VOCAB, embed_dim=EMBEDDING_DIM)]

# Item 特征
item_cols = [
    SparseFeature("news_id_idx", vocab_size=FIXED_NEWS_VOCAB, embed_dim=EMBEDDING_DIM),
    SparseFeature("cat_idx", vocab_size=vocab_size_cat, embed_dim=EMBEDDING_DIM),        
    SparseFeature("subcat_idx", vocab_size=vocab_size_subcat, embed_dim=EMBEDDING_DIM)
]

# History 特征
history_cols = [
    SequenceFeature("history_seq", vocab_size=FIXED_NEWS_VOCAB, embed_dim=EMBEDDING_DIM, pooling="concat", shared_with="news_id_idx"),
    SequenceFeature("hist_cat_seq", vocab_size=vocab_size_cat, embed_dim=EMBEDDING_DIM, pooling="concat", shared_with="cat_idx"),
    SequenceFeature("hist_subcat_seq", vocab_size=vocab_size_subcat, embed_dim=EMBEDDING_DIM, pooling="concat", shared_with="subcat_idx")
]

features = user_cols + item_cols

# ==========================================
# 3. 加载模型
# ==========================================
print("正在加载最佳模型...")
model_din = DIN(
    features=features,
    history_features=history_cols,
    target_features=item_cols,
    mlp_params={"dims": [256, 128], "dropout": 0.3},
    attention_mlp_params={"dims": [64, 32]} 
)

try:
    model_din.load_state_dict(torch.load('din_best_model.pth', map_location=device))
    model_din.to(device)
    model_din.eval()
    print("✅ 模型权重加载成功！(骨架已强行适配)")
except RuntimeError as e:
    print(f"❌ 依然失败: {e}")
    raise

# ==========================================
# 4. 处理测试集 (带越界保护)
# ==========================================
# 定义处理函数
def process_test_data(df):
    samples = []
    for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing Test Set"):
        user_id = str(row['user_id'])
        imp_id = str(row['impression_id'])
        hist = str(row['history']).split() if pd.notna(row['history']) else ['N0']
        hist_str = " ".join(hist)
        impressions = str(row['impressions']).split()
        for i, news_id in enumerate(impressions):
            samples.append([user_id, hist_str, news_id, imp_id, i])
    return pd.DataFrame(samples, columns=['user_id', 'history_str', 'news_id', 'impression_id', 'imp_index'])

# 读取测试集
if 'test_behaviors' not in globals():
    print("读取测试集原始文件...")
    try:
        test_behaviors = pd.read_csv('data/MINDlarge_test/behaviors.tsv', sep='\t', header=None, names=behaviors_cols)
    except:
        # 尝试备用路径
        test_behaviors = pd.read_csv('data/MINDlarge_test/behaviors.tsv', sep='\t', header=None, names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

print("构建测试集样本...")
test_df = process_test_data(test_behaviors)

# --- 特征映射与越界清洗 ---
print("特征映射中 (启用安全模式)...")

# User 映射
# 即使内存里有 lbe_user，它的映射关系可能和训练时不完全一致，但 ID 顺序通常不变
# 我们假设 lbe_user 是存在的。如果不存在，这里会报错，你需要先跑上面的恢复代码
user_map = dict(zip(lbe_user.classes_, range(len(lbe_user.classes_))))
test_df['user_id_idx'] = test_df['user_id'].map(user_map).fillna(0).astype(int)

# 【越界保护】如果现在的 ID 超过了模型承受范围 (750434)，强制设为 0 (UNK)
test_df['user_id_idx'] = test_df['user_id_idx'].apply(lambda x: x if x < FIXED_USER_VOCAB else 0)

# News ID 映射
test_df['news_id_idx'] = test_df['news_id'].apply(lambda x: news_map.get(x, 0))
# 【越界保护】
test_df['news_id_idx'] = test_df['news_id_idx'].apply(lambda x: x if x < FIXED_NEWS_VOCAB else 0)

# Cat / SubCat
test_df['cat_idx'] = test_df['news_id'].apply(lambda x: news2cat.get(x, 0))
test_df['subcat_idx'] = test_df['news_id'].apply(lambda x: news2subcat.get(x, 0))

# 历史序列映射
def get_seq_ids_safe(s, mapping, max_vocab):
    # 映射并进行安全检查
    ids = [mapping.get(x, 0) for x in s.split()]
    return [i if i < max_vocab else 0 for i in ids]

test_df['hist_idx_list'] = test_df['history_str'].apply(lambda x: get_seq_ids_safe(x, news_map, FIXED_NEWS_VOCAB))
test_df['hist_cat_list'] = test_df['history_str'].apply(lambda x: get_seq_ids_safe(x, news2cat, 999999)) # Cat通常不会越界
test_df['hist_subcat_list'] = test_df['history_str'].apply(lambda x: get_seq_ids_safe(x, news2subcat, 999999))

# Padding
print("Padding序列...")
SEQ_LEN = 20
def pad(series):
    return list(pad_sequences(series.tolist(), maxlen=SEQ_LEN, padding='post', truncating='post', value=0))

test_df['history_seq'] = pad(test_df['hist_idx_list'])
test_df['hist_cat_seq'] = pad(test_df['hist_cat_list'])
test_df['hist_subcat_seq'] = pad(test_df['hist_subcat_list'])

# ==========================================
# 5. 预测并打包
# ==========================================
input_cols = ['user_id_idx', 'news_id_idx', 'cat_idx', 'subcat_idx', 
              'history_seq', 'hist_cat_seq', 'hist_subcat_seq']

test_dg = DataGenerator(x=test_df[input_cols], y=test_df['impression_id'])
test_loader, _, _ = test_dg.generate_dataloader(x_val=test_df[input_cols], y_val=test_df['impression_id'], batch_size=4096, shuffle=False)

print("正在预测...")
y_pred = []
with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Predicting"):
        x = {k: v.to(device) for k, v in x.items()}
        pred = model_din(x).cpu().data.numpy().squeeze()
        y_pred.extend(pred)

test_df['score'] = y_pred

print("生成 prediction.txt ...")
lines = []
for imp_id, group in tqdm(test_df.groupby('impression_id'), desc="Grouping"):
    group = group.sort_values('imp_index')
    scores = group['score'].values
    ranks = rankdata(-scores, method='ordinal').astype(int)
    rank_str = '[' + ','.join(map(str, ranks)) + ']'
    lines.append(f"{imp_id} {rank_str}")

with open('prediction.txt', 'w') as f:
    for line in lines:
        f.write(line + '\n')

print("正在压缩...")
with zipfile.ZipFile('prediction.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.write('prediction.txt')

print("🎉 恭喜！问题解决，prediction.zip 已生成！")

🚀 强制对齐模型骨架: User=750434, News=101288
正在加载最佳模型...
✅ 模型权重加载成功！(骨架已强行适配)
读取测试集原始文件...
构建测试集样本...


Processing Test Set: 100%|█████████| 2370727/2370727 [01:45<00:00, 22543.78it/s]


特征映射中 (启用安全模式)...


In [17]:
import torch

model_path = 'din_best_model.pth'

print(f"🔍 正在检查模型文件: {model_path} ...")

try:
    # 加载模型参数字典 (映射到 CPU 读取)
    state_dict = torch.load(model_path, map_location='cpu')
    
    # 查找 news_id_idx 的权重
    # 通常 key 是 'embedding.embed_dict.news_id_idx.weight'
    target_key = None
    for key in state_dict.keys():
        if 'news_id_idx' in key and 'weight' in key:
            target_key = key
            break
            
    if target_key:
        weight_shape = state_dict[target_key].shape
        actual_dim = weight_shape[1] # [vocab_size, embedding_dim]
        
        print("="*40)
        print(f"✅ 检测结果出炉！")
        print(f"权重名称: {target_key}")
        print(f"权重形状: {weight_shape}")
        print(f"👉 你的模型真实维度 (EMBEDDING_DIM) 是: 【{actual_dim}】")
        print("="*40)
        
        # 自动生成建议
        if actual_dim == 32:
            print("请在后续代码中设置: EMBEDDING_DIM = 32")
        elif actual_dim == 64:
            print("请在后续代码中设置: EMBEDDING_DIM = 64")
    else:
        print("❌ 未找到 News ID 的 Embedding 层，请检查模型文件是否完整。")
        # 打印所有 keys 帮你排查
        print("所有 Keys:", state_dict.keys())

except FileNotFoundError:
    print(f"❌ 找不到文件: {model_path}。请确认文件名是否正确。")
except Exception as e:
    print(f"❌ 读取出错: {e}")

🔍 正在检查模型文件: din_best_model.pth ...
✅ 检测结果出炉！
权重名称: embedding.embed_dict.news_id_idx.weight
权重形状: torch.Size([101288, 32])
👉 你的模型真实维度 (EMBEDDING_DIM) 是: 【32】
请在后续代码中设置: EMBEDDING_DIM = 32
